## 矩阵分解

目标函数：


$$arg \min _{P,Q,b} \sum_{u,i}\Vert R_{ui} - \hat R_{ui} \Vert ^2 + \lambda (\Vert P \Vert ^{2} + \Vert Q\Vert ^2 + b_{u}^2 + b_{i} ^2)  $$

$$P \in R^{m \times k},Q \in R^{n \times k},\hat R_{ui} = p_{u}q_{i}^{T} + b_{u} + b_{i}$$

评价指标：


$$RMSE = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (y_i - \hat y_i)^{2}}$$

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.model_selection import train_test_split

In [3]:
path = '../data/ml-100k/u.data'

In [4]:
BATCH_SIZE = 512

In [5]:
data = pd.read_csv(path,sep='\t',header=None,names=['user_id', 'item_id', 'rating', 'timestamp'] )

In [6]:
num_users = data.user_id.unique().shape[0]+1
num_items = data.item_id.unique().shape[0]+1
num_factors=30

In [7]:
num_users,num_items

(944, 1683)

In [8]:
data.user_id.max(),data.item_id.max()

(943, 1682)

In [9]:
data.head(5)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
X_train, X_test, y_train, y_test = train_test_split(data[['user_id', 'item_id']].values, data['rating'].values, test_size=0.1, random_state=2020)

In [11]:
X_train = torch.tensor(X_train,dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [12]:
X_train.shape

torch.Size([90000, 2])

In [13]:
train_loader = Data.DataLoader(
    dataset = Data.TensorDataset(X_train,y_train),
    batch_size = BATCH_SIZE,
    shuffle=True,
)
test_loader = Data.DataLoader(
    dataset = Data.TensorDataset(X_test,y_test),
    batch_size = BATCH_SIZE,
    shuffle=True,
)

In [14]:
class MF(nn.Module):
    def __init__(self, num_factors, num_users, num_items):
        super(MF, self).__init__()
        self.P = nn.Embedding(num_embeddings=num_users, embedding_dim=num_factors)
        self.Q = nn.Embedding(num_embeddings=num_items, embedding_dim=num_factors)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)
        
    def forward(self, data):
        user_id, item_id = data[:,0], data[:,1]
        P_u = self.P(user_id)
        Q_i = self.Q(item_id)
        b_u = self.user_bias(user_id)
        b_i = self.item_bias(item_id)
        outputs = ((P_u*Q_i).sum(dim=1)) + b_u.squeeze() + b_i.squeeze()
        return outputs.flatten()

In [15]:
model = MF(num_factors=num_factors, num_users=num_users, num_items=num_items)

In [16]:
def RMSE(y, y_):
    loss = ((y-y_)**2).sum()
    loss = loss/len(y)
    return loss**0.5

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=0.0001)
loss_func = RMSE

In [18]:
for epoch in range(30) :
    for step, (tx, ty) in enumerate(train_loader) :
        #print(tx.shape)
        output=model(tx)
        
        train_loss = loss_func(output, ty)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if step % 300 == 0 :
            with torch.no_grad(): 
                y = model(X_test)
                test_loss = loss_func(y, y_test)
                print('Epoch : {}|train_loss : {:.4f}| test_loss :{:.4f}'.format(epoch, train_loss.item(), test_loss.item()))

Epoch : 0|train_loss : 6.856884956359863| test_loss :6.66071081161499
Epoch : 1|train_loss : 1.4677066802978516| test_loss :1.9263982772827148
Epoch : 2|train_loss : 0.8249659538269043| test_loss :1.094244360923767
Epoch : 3|train_loss : 0.8286493420600891| test_loss :1.0006293058395386
Epoch : 4|train_loss : 0.7758222222328186| test_loss :0.9718416929244995
Epoch : 5|train_loss : 0.7339785695075989| test_loss :0.9615576267242432
Epoch : 6|train_loss : 0.7579453587532043| test_loss :0.9613692760467529
Epoch : 7|train_loss : 0.7265439629554749| test_loss :0.9685827493667603
Epoch : 8|train_loss : 0.6972488164901733| test_loss :0.9689694046974182
Epoch : 9|train_loss : 0.7354786396026611| test_loss :0.9662456512451172
Epoch : 10|train_loss : 0.6774872541427612| test_loss :0.9644127488136292
Epoch : 11|train_loss : 0.7153388857841492| test_loss :0.9741464257240295
Epoch : 12|train_loss : 0.6791142225265503| test_loss :0.9709009528160095
Epoch : 13|train_loss : 0.6705185174942017| test_los

In [19]:
y = model(X_test)

In [20]:
y

tensor([4.2477, 2.3422, 4.7106,  ..., 3.8516, 3.5220, 3.1259],
       grad_fn=<AddBackward0>)

In [21]:
y_test

tensor([5., 1., 3.,  ..., 5., 2., 2.])

In [22]:
#def weights_init(model):
#    m_name = model.__class__.__name__
#    if m_name.find('Embedding')!= -1 :
#        model.weight.data.normal_(0.0,0.01)
#model.apply(weights_init)